In [1]:
import sqlite3
import configparser


# your .tsv for import file
file_name = "tsv/dataset_part3.tsv"

In [2]:
query_insert = """
        INSERT INTO reddit_bots VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
"""

In [3]:
config = configparser.ConfigParser()
config.read('config.ini')
connection = sqlite3.connect(config['DATABASE']['FILE'])
cursor = connection.cursor()

In [4]:
data = []
unique_account = set()

with open(file_name, 'r', encoding="utf-8") as file:
    is_first_line = True
    file.readline().split("\t")
    for line in file:
        if is_first_line:
            is_first_line = False
            continue
        len_unique_account = len(unique_account)
        line = line.replace('\n','')
        line = line.split("\t")

        line[2] = line[2].strip()

        user = line[2]
        user = user.replace("^s","")
        user = user.replace("^b","")
        user = user.replace("/submitted","")
        user = user.replace("'s","")
        user = user.replace("\\","")
        user = user.replace("/overview","")
        user = user.replace("u/","")
        user = user.replace("/posts","")
        user = user.replace("https://www.reddit.com/user/","")
        user = user.replace("https://old.reddit.com/user/","")
        if user.find("/comments") > -1:
            user = user[:(user.find("/comments"))]

        if user.find("|") > -1:
            user = user[:(user.find("|"))]


        unique_account.add(user)
        is_duplicate = True

        if len(unique_account) == len_unique_account:
            continue
            is_duplicate = False

        if '/' in user:
            continue
        
        annotator = line[1] 
        if line[1] == None or line[1] == '':
            annotator = None

        data = (annotator, user, None, None, None, None, None, None, None, None, None, None, None, None, None)
        cursor.execute(query_insert,(data) )
        connection.commit()

In [5]:
cursor.close()
connection.close()